In [1]:
!pip install pandas
!pip install openpyxl


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


IPLOM

In [2]:
from IPLoM2 import LogParser

path = 'sample_log.txt'
log_format = " <Month> <Date> <Time> <Level> <Component>(\[<PID>\])?: <Content>"
ct = 0.35
lower_bound = 0
output_dir = 'output'
parser = LogParser(path, log_format, ct, lower_bound, output_dir)
parser.parse_logs()

^(?P<Month>.*?)\s+(?P<Date>.*?)\s+(?P<Time>.*?)\s+(?P<Level>.*?)\s+(?P<Component>.*?)(\[(?P<PID>.*?)\])?:\s+(?P<Content>.*?)$
     Month Date  ... LineId   EventId
0      Jun   14  ...      1  mc3gat6y
1      Jun   14  ...      2  mc3gat6y
2      Jun   15  ...      3  mc3gat6y
3      Jun   15  ...      4  mc3gat6y
4      Jun   15  ...      5  mc3gat6y
...    ...  ...  ...    ...       ...
1995   Jul   27  ...      2  kn2mfipl
1996   Jul   27  ...      1  tnddwjro
1997   Jul   27  ...      1  21ma8rjq
1998   Jul   27  ...      1  obkq3rlh
1999   Jul   27  ...      2  obkq3rlh

[2000 rows x 9 columns]


Import Required Modules

In [5]:
import pandas as pd
import os

Open the CSV File and filter the requied coloumns

In [6]:
cwd = os.getcwd()
filepath = cwd + '\\output\\clusters.csv'
print(filepath)
logs = pd.read_csv( filepath, header=0)
# display(logs)
full_logs = logs[['EventId', 'Content','PID']]
display(full_logs)


c:\Users\17787\Documents\SFU\SEM2\LAB\Project\output\clusters.csv


,EventId,Content,PID
0,mc3gat6y,authentication failure; logname= uid=0 euid=0 ...,19939.0
1,mc3gat6y,authentication failure; logname= uid=0 euid=0 ...,19937.0
2,mc3gat6y,authentication failure; logname= uid=0 euid=0 ...,23397.0
3,mc3gat6y,authentication failure; logname= uid=0 euid=0 ...,23395.0
4,mc3gat6y,authentication failure; logname= uid=0 euid=0 ...,23404.0
...,...,...,...
1995,kn2mfipl,pci_hotplug: PCI Hot Plug PCI Core version: 0.5,NaN
1996,tnddwjro,apm: BIOS version 1.2 Flags 0x03 (Driver versi...,NaN
1997,21ma8rjq,Setting network parameters: succeeded,NaN
1998,obkq3rlh,isapnp: No Plug & Play device found,NaN


Counting the number of events

In [7]:
# count the number of occurrences of each fruit in the 'EventId' column
event_counts= logs['EventId'].value_counts()

# create a new DataFrame with the counts
counts_df = pd.DataFrame({'EventId': event_counts.index, 'Occurences': event_counts.values})

# merge count data frame with original data frame
logs = pd.merge(full_logs, counts_df, on='EventId')

# print resulting data frame



COUNT THE NUMBER OF OCCURENCES OF EACH EVENT

In [8]:
display(logs)

,EventId,Content,PID,Occurences
0,mc3gat6y,authentication failure; logname= uid=0 euid=0 ...,19939.0,118
1,mc3gat6y,authentication failure; logname= uid=0 euid=0 ...,19937.0,118
2,mc3gat6y,authentication failure; logname= uid=0 euid=0 ...,23397.0,118
3,mc3gat6y,authentication failure; logname= uid=0 euid=0 ...,23395.0,118
4,mc3gat6y,authentication failure; logname= uid=0 euid=0 ...,23404.0,118
...,...,...,...,...
1995,kn2mfipl,pci_hotplug: PCI Hot Plug PCI Core version: 0.5,NaN,2
1996,tnddwjro,apm: BIOS version 1.2 Flags 0x03 (Driver versi...,NaN,1
1997,21ma8rjq,Setting network parameters: succeeded,NaN,1
1998,obkq3rlh,isapnp: No Plug & Play device found,NaN,2


Lemmatization

Spacy

In [9]:
!pip install spacy
!python -m spacy download en_core_web_sm


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     --------------------------------------- 12.8/12.8 MB 38.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import spacy

# load the spaCy English language model
nlp = spacy.load("en_core_web_sm")

# define a function to lemmatize a sentence using spaCy
def lemmatize_sentence(sentence):
    # parse the input sentence with spaCy
    doc = nlp(sentence)
    
    # lemmatize each token in the parsed sentence
    lemmas = [token.lemma_ for token in doc]
    
    # join the lemmas back into a sentence
    lemmatized_sentence = ' '.join(lemmas)
    
    return lemmatized_sentence


# apply the lemmatize_sentence function to the 'Content' column
logs['Lemmatized Content'] = logs['Content'].apply(lambda x: lemmatize_sentence(x))

# print the resulting DataFrame



SHOW THE LEMMATIZED CONTENT

In [12]:
display(logs)
logs.to_csv('lemmatized.csv',index=False)

,EventId,Content,PID,Occurences,Lemmatized Content
0,mc3gat6y,authentication failure; logname= uid=0 euid=0 ...,19939.0,118,authentication failure ; logname= uid=0 euid=0...
1,mc3gat6y,authentication failure; logname= uid=0 euid=0 ...,19937.0,118,authentication failure ; logname= uid=0 euid=0...
2,mc3gat6y,authentication failure; logname= uid=0 euid=0 ...,23397.0,118,authentication failure ; logname= uid=0 euid=0...
3,mc3gat6y,authentication failure; logname= uid=0 euid=0 ...,23395.0,118,authentication failure ; logname= uid=0 euid=0...
4,mc3gat6y,authentication failure; logname= uid=0 euid=0 ...,23404.0,118,authentication failure ; logname= uid=0 euid=0...
...,...,...,...,...,...
1995,kn2mfipl,pci_hotplug: PCI Hot Plug PCI Core version: 0.5,NaN,2,pci_hotplug : pci Hot Plug pci core version : 0.5
1996,tnddwjro,apm: BIOS version 1.2 Flags 0x03 (Driver versi...,NaN,1,apm : BIOS version 1.2 Flags 0x03 ( Driver ver...
1997,21ma8rjq,Setting network parameters: succeeded,NaN,1,set network parameter : succeed
1998,obkq3rlh,isapnp: No Plug & Play device found,NaN,2,isapnp : No Plug & play device find


Check For Failures

In [13]:
def failure(text):
    if 'failure' in text.lower():
        return 'Failure '
    elif 'fatal' in text.lower():
        return 'Failure'
    elif 'severe' in text.lower():
        return 'Failure'
    elif 'critical' in text.lower():
        return 'Failure'
    elif 'alert' in text.lower():
        return 'Failure'
    elif 'emergency' in text.lower():
        return 'Failure'
    elif 'unknown' in text.lower():
        return 'Failure'
    elif 'fail' in text.lower():
        return 'Failure'
    elif 'error' in text.lower():
        return 'Error'
    elif 'fault' in text.lower():
        return 'Error'
    elif 'notice' in text.lower():
        return 'Warning'
    elif 'warn' in text.lower():
        return 'Warning'
    elif 'finest' in text.lower():
        return 'Debug'
    elif 'verbose' in text.lower():
        return 'Debug'
    elif 'finer' in text.lower():
        return 'Debug'
    elif 'trace' in text.lower():
        return 'Debug'
    elif 'basic' in text.lower():
        return 'Debug'
    elif 'fine' in text.lower():
        return 'Debug'
    elif 'config' in text.lower():
        return 'Debug'
    elif 'info' in text.lower():
        return 'Info'
    elif 'success' in text.lower():
        return 'Info'
    
    
    
    
logs["Tag"] = logs["Lemmatized Content"].apply(failure)
display(logs)


,EventId,Content,PID,Occurences,Lemmatized Content,Tag
0,mc3gat6y,authentication failure; logname= uid=0 euid=0 ...,19939.0,118,authentication failure ; logname= uid=0 euid=0...,Failure
1,mc3gat6y,authentication failure; logname= uid=0 euid=0 ...,19937.0,118,authentication failure ; logname= uid=0 euid=0...,Failure
2,mc3gat6y,authentication failure; logname= uid=0 euid=0 ...,23397.0,118,authentication failure ; logname= uid=0 euid=0...,Failure
3,mc3gat6y,authentication failure; logname= uid=0 euid=0 ...,23395.0,118,authentication failure ; logname= uid=0 euid=0...,Failure
4,mc3gat6y,authentication failure; logname= uid=0 euid=0 ...,23404.0,118,authentication failure ; logname= uid=0 euid=0...,Failure
...,...,...,...,...,...,...
1995,kn2mfipl,pci_hotplug: PCI Hot Plug PCI Core version: 0.5,NaN,2,pci_hotplug : pci Hot Plug pci core version : 0.5,None
1996,tnddwjro,apm: BIOS version 1.2 Flags 0x03 (Driver versi...,NaN,1,apm : BIOS version 1.2 Flags 0x03 ( Driver ver...,None
1997,21ma8rjq,Setting network parameters: succeeded,NaN,1,set network parameter : succeed,None
1998,obkq3rlh,isapnp: No Plug & Play device found,NaN,2,isapnp : No Plug & play device find,None


In [14]:
def tag_info(text):
    if  text is None:
        return 'Info'
    else:
        return text

logs["Tag"] = logs["Tag"].apply(tag_info)
logs.to_csv('tags.csv',index=False)

DISPLAY ALL TAGS

In [15]:
display(logs)

,EventId,Content,PID,Occurences,Lemmatized Content,Tag
0,mc3gat6y,authentication failure; logname= uid=0 euid=0 ...,19939.0,118,authentication failure ; logname= uid=0 euid=0...,Failure
1,mc3gat6y,authentication failure; logname= uid=0 euid=0 ...,19937.0,118,authentication failure ; logname= uid=0 euid=0...,Failure
2,mc3gat6y,authentication failure; logname= uid=0 euid=0 ...,23397.0,118,authentication failure ; logname= uid=0 euid=0...,Failure
3,mc3gat6y,authentication failure; logname= uid=0 euid=0 ...,23395.0,118,authentication failure ; logname= uid=0 euid=0...,Failure
4,mc3gat6y,authentication failure; logname= uid=0 euid=0 ...,23404.0,118,authentication failure ; logname= uid=0 euid=0...,Failure
...,...,...,...,...,...,...
1995,kn2mfipl,pci_hotplug: PCI Hot Plug PCI Core version: 0.5,NaN,2,pci_hotplug : pci Hot Plug pci core version : 0.5,Info
1996,tnddwjro,apm: BIOS version 1.2 Flags 0x03 (Driver versi...,NaN,1,apm : BIOS version 1.2 Flags 0x03 ( Driver ver...,Info
1997,21ma8rjq,Setting network parameters: succeeded,NaN,1,set network parameter : succeed,Info
1998,obkq3rlh,isapnp: No Plug & Play device found,NaN,2,isapnp : No Plug & play device find,Info


Sort on the basis of each category

Check for Tags

In [16]:

tags = pd.read_csv("tags.csv", header=0)
# display(logs)
Failure = tags[tags['Tag'] == 'Failure']
Failure_logs = Failure.sort_values(by=["Occurences"], ascending=False)
# del logs
Info = tags[tags['Tag'] == 'Info']
Info_logs = Info.sort_values(by=["Occurences"], ascending=False)
Error_log = tags[tags['Tag'] == 'Error']
Error_logs = Error_log.sort_values(by=["Occurences"], ascending=False)
Debug_log = tags[tags['Tag'] == 'Debug']
Debug_logs = Debug_log.sort_values(by=["Occurences"], ascending=False)
Warning_log = tags[tags['Tag'] == 'Warning']
Warning_logs = Warning_log.sort_values(by=["Occurences"], ascending=False)
# display(Failure_logs)
# display(Info_logs)
# display(Error_logs)
# display(Warning_logs)
# display(Debug_logs)

sorted_logs = pd.concat([Failure_logs,Error_logs,Warning_logs,Debug_logs,Info_logs], axis=0)
sorted_logs = sorted_logs.reset_index()
display(sorted_logs)
sorted_logs.to_csv('Classified_logs.csv', index=False)
unique_logs = sorted_logs.drop_duplicates(subset='EventId', keep='first')
unique_logs = unique_logs.reset_index()
display(unique_logs)
unique_logs.to_csv('Unique_failures.csv',index=False)


,index,EventId,Content,PID,Occurences,Lemmatized Content,Tag
0,118,9ioh3qs3,check pass; user unknown,19937.0,117,check pass ; user unknown,Failure
1,192,9ioh3qs3,check pass; user unknown,18559.0,117,check pass ; user unknown,Failure
2,204,9ioh3qs3,check pass; user unknown,24574.0,117,check pass ; user unknown,Failure
3,203,9ioh3qs3,check pass; user unknown,24573.0,117,check pass ; user unknown,Failure
4,202,9ioh3qs3,check pass; user unknown,24590.0,117,check pass ; user unknown,Failure
...,...,...,...,...,...,...,...
1504,1903,das9pdwk,imps2: Auto-detected intellimouse PS/2,2094.0,1,imps2 : auto - detect intellimouse ps/2,Info
1505,1902,qvwapyk0,*** info [mice.c(1766)]:,2094.0,1,* * * info [ mice.c(1766 ) ] :,Info
1506,1996,tnddwjro,apm: BIOS version 1.2 Flags 0x03 (Driver versi...,NaN,1,apm : BIOS version 1.2 Flags 0x03 ( Driver ver...,Info
1507,1997,21ma8rjq,Setting network parameters: succeeded,NaN,1,set network parameter : succeed,Info


,level_0,index,EventId,Content,PID,Occurences,Lemmatized Content,Tag
0,0,118,9ioh3qs3,check pass; user unknown,19937.0,117,check pass ; user unknown,Failure
1,117,877,ttmqhg3f,ALERT exited abnormally with [1],NaN,51,ALERT exit abnormally with [ 1 ],Failure
2,160,1879,dfozxvae,Kerberos authentication failed,19277.0,23,kerberos authentication fail,Failure
3,183,1896,gwc8pjc6,Authentication failed from 163.27.187.39 (163....,19280.0,15,authentication fail from 163.27.187.39 ( 163.2...,Failure
4,198,1856,iodri1en,Authentication failed from 163.27.187.39 (163....,19272.0,8,authentication fail from 163.27.187.39 ( 163.2...,Failure
5,206,1813,lwrxywbz,User unknown timed out after 900 seconds at Sa...,31277.0,1,User unknown time out after 900 second at Sat ...,Failure
6,207,1987,rv0gmki8,There is already a security framework initiali...,NaN,1,there be already a security framework initiali...,Failure
7,208,1936,ib6jvi1h,warning: can't get client address: Connection ...,26482.0,2,warning : can not get client address : connect...,Warning
8,211,1300,luzwwqoc,connection from 206.196.21.129 (host129.206.19...,24088.0,909,connection from 206.196.21.129 ( host129.206.1...,Info
9,1120,607,cnjorbbn,session opened for user cyrus by (uid=0),21416.0,123,session open for user cyrus by ( uid=0 ),Info


ARRANGE ACCORDING TO THE TAGS AND SHOW THE UNIQUE LOGS

In [17]:
display(sorted_logs)
display(unique_logs)

,index,EventId,Content,PID,Occurences,Lemmatized Content,Tag
0,118,9ioh3qs3,check pass; user unknown,19937.0,117,check pass ; user unknown,Failure
1,192,9ioh3qs3,check pass; user unknown,18559.0,117,check pass ; user unknown,Failure
2,204,9ioh3qs3,check pass; user unknown,24574.0,117,check pass ; user unknown,Failure
3,203,9ioh3qs3,check pass; user unknown,24573.0,117,check pass ; user unknown,Failure
4,202,9ioh3qs3,check pass; user unknown,24590.0,117,check pass ; user unknown,Failure
...,...,...,...,...,...,...,...
1504,1903,das9pdwk,imps2: Auto-detected intellimouse PS/2,2094.0,1,imps2 : auto - detect intellimouse ps/2,Info
1505,1902,qvwapyk0,*** info [mice.c(1766)]:,2094.0,1,* * * info [ mice.c(1766 ) ] :,Info
1506,1996,tnddwjro,apm: BIOS version 1.2 Flags 0x03 (Driver versi...,NaN,1,apm : BIOS version 1.2 Flags 0x03 ( Driver ver...,Info
1507,1997,21ma8rjq,Setting network parameters: succeeded,NaN,1,set network parameter : succeed,Info


,level_0,index,EventId,Content,PID,Occurences,Lemmatized Content,Tag
0,0,118,9ioh3qs3,check pass; user unknown,19937.0,117,check pass ; user unknown,Failure
1,117,877,ttmqhg3f,ALERT exited abnormally with [1],NaN,51,ALERT exit abnormally with [ 1 ],Failure
2,160,1879,dfozxvae,Kerberos authentication failed,19277.0,23,kerberos authentication fail,Failure
3,183,1896,gwc8pjc6,Authentication failed from 163.27.187.39 (163....,19280.0,15,authentication fail from 163.27.187.39 ( 163.2...,Failure
4,198,1856,iodri1en,Authentication failed from 163.27.187.39 (163....,19272.0,8,authentication fail from 163.27.187.39 ( 163.2...,Failure
5,206,1813,lwrxywbz,User unknown timed out after 900 seconds at Sa...,31277.0,1,User unknown time out after 900 second at Sat ...,Failure
6,207,1987,rv0gmki8,There is already a security framework initiali...,NaN,1,there be already a security framework initiali...,Failure
7,208,1936,ib6jvi1h,warning: can't get client address: Connection ...,26482.0,2,warning : can not get client address : connect...,Warning
8,211,1300,luzwwqoc,connection from 206.196.21.129 (host129.206.19...,24088.0,909,connection from 206.196.21.129 ( host129.206.1...,Info
9,1120,607,cnjorbbn,session opened for user cyrus by (uid=0),21416.0,123,session open for user cyrus by ( uid=0 ),Info
